In [ ]:
import requests as rq
import bs4 as bs4
import time
import json
import pandas as pd
import numpy as np
import math

# Parte 1: Editoras

In [ ]:
# Url onde estão todas as editoras

url_editoras = 'https://www.skoob.com.br/editoras/'

In [ ]:
# Fazendo request do link e transformando em um objeto do bs4

response = rq.get(url_editoras)
pagina_html = bs4.BeautifulSoup(response.text)

In [ ]:
# Procurando tags

tags = pagina_html.findAll(id="resultadoBusca-")
editoras = tags[0].findAll('strong')

lista_editoras = []
lista_links = []

for x in editoras:
    if x.find('a') != None:
        editora = x.get_text()
        lista_editoras.append(editora)
        
        link = x.find('a').get('href')
        lista_links.append(link)
        
livros_editora = []
numero_livros = tags[0].findAll('span')

for livros in range(1, len(numero_livros), 8):
    n_livros = numero_livros[livros].get_text().split()[0]
    n_livros = n_livros.replace('.', '')
    livros_editora.append(n_livros)

In [ ]:
# Criando dataframe que armazenará os dados

df = pd.DataFrame()
df['editora'] = lista_editoras
df['link'] = lista_links
df['qtde_livros'] = livros_editora

In [ ]:
# Função para ser aplicada em cada linha do dataframe na coluna código
# Parâmetro x: string -> link da editora
# Irá retornar o código da editora

def codigo(x):
    return x.split('/')[-1].split('-')[0]

codigos = df_editoras['link'].apply(codigo)
df['codigo'] = codigos

In [ ]:
# Dataframe com as editoras
df.to_csv('./editoras_link.csv', index=False)

In [ ]:
df

# Parte 2: Livros das Editoras


Os livros de cada editora estão no link : "https://www.skoob.com.br/editora/livros/codigoDeCadaEditora"  
É necessário o código de cada editora para entrar na página dos livros. Os códigos são os números da coluna link do df_editoras

In [ ]:
lista_livros = []
lista_livros_link = []

for index, row in df.iterrows():
    print('--------------------------')
    print(df['editora'][index])
    
    
    url_base = 'https://www.skoob.com.br/editora/livros/{}/mpage:{}'
    
    # Número de páginas com livros 
    n_page = math.ceil(int(df['qtde_livros'][index])/40) + 1
    
    for x in range(1, n_page):
        print(x)
        url_livros = url_base.format(df['codigo'][index], x)
        
        response = rq.get(url_livros)
        pagina_html = bs4.BeautifulSoup(response.text)
        
        lista_div = pagina_html.findAll("div", {"class": "box_livro"})
        
        for x in lista_div:
            titulo = x.find('h3').get_text()
            link = x.find('a').get('href')
            
            lista_livros.append(titulo)
            lista_livros_link.append(link)
        
        time.sleep(1)


In [ ]:
len(lista_livros), len(lista_livros_link)

In [ ]:
df = pd.DataFrame()
df['livros'] = lista_livros
df['link'] = lista_livros_link

In [156]:
# Dataframe com as editoras
df.to_csv('./livros_link.csv', index=False)

In [157]:
df

,livros,link
0,Orçamento sem falhas,/orcamento-sem-falhas-11677985ed11693528.html
1,Minha Sombria Vanessa,/minha-sombria-vanessa-1022974ed1025216.html
2,Recursão,/recursao-980390ed1030802.html
3,"M, o Filho do Século",/m-o-filho-do-seculo-1028489ed1030829.html
4,Oblivion Song: Entre Dois Mundos,/oblivion-song-entre-dois-mundos-1030237ed1032...
...,...,...
11994,Poemas completos de Alberto Caeiro,/poemas-completos-de-alberto-caeiro-1069ed1420...
11995,Poesia de Álvaro de Campos,/poesia-de-alvaro-de-campos-26706ed55518.html
11996,Ben-Hur,/ben-hur-538ed23492.html
11997,Folhas de Relva,/folhas-de-relva-274ed376.html


## Editora AllBook

In [ ]:
url_livros = 'https://www.skoob.com.br/editora/livros/11/mpage:34'

In [ ]:
response = rq.get(url_livros)
pagina_html = bs4.BeautifulSoup(response.text)

In [ ]:
lista_div = pagina_html.findAll("div", {"class": "box_livro"})

    

In [ ]:
for x in lista_div:
    titulo = x.find('h3').get_text()
    link = x.find('a').get('href')

In [ ]:
lista_div